In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib
rcParams.update({'figure.autolayout': True})
rcParams['svg.fonttype'] = 'none'
rcParams['font.size']=10

from keras.models import load_model

import pybedtools

Using Theano backend.
Can not use cuDNN on context None: cannot compile with cuDNN. We got this error:
b'/tmp/try_flags_kfyzwv6g.c:4:10: fatal error: cudnn.h: No such file or directory\n #include <cudnn.h>\n          ^~~~~~~~~\ncompilation terminated.\n'
Preallocating 10619/11178 Mb (0.950000) on cuda
Mapped name None to device cuda: GeForce GTX 1080 Ti (0000:B1:00.0)


In [2]:
colorsDict = {"Microglia":'#6b97ff',
              "Monocyte": '#ff6b97',
              "Neuron": '#97ff6b'}

In [3]:
microglia_regression = load_model("/projects/pfenninggroup/machineLearningForComputationalBiology/eramamur_stuff/ml_gosselin_microglia/model_7.hdf5", compile = False)
monocyte_regression = load_model("/projects/pfenninggroup/machineLearningForComputationalBiology/eramamur_stuff/ml_monocyte/model_8.hdf5", compile = False)
neuron_regression = load_model("/projects/pfenninggroup/machineLearningForComputationalBiology/eramamur_stuff/ml_fullard_neun/model_2.hdf5", compile = False)

In [4]:
bulk_models_list = {"Microglia": microglia_regression,
                    "Monocyte": monocyte_regression,
                    "Neuron": neuron_regression}

In [5]:
rootPath = "/".join(["/projects",
                     "pfenninggroup",
                     "machineLearningForComputationalBiology",
                     "eramamur_stuff",
                     "ad_variants_processing",
                     "common_variants"])

In [23]:
region_bed = pybedtools.BedTool(os.path.join(rootPath,"alpk2_region_1_50kb.bed"))
cell_type_beds = {"Monocyte":pybedtools.BedTool(os.path.join("/projects",
                                                              "pfenninggroup",
                                                              "machineLearningForComputationalBiology",
                                                              "eramamur_stuff",
                                                              "gwas_enrichments",
                                                              "roadmap_dnase",
                                                              "encode_monocyte_dhs_optimal_peak_liftover_hg38_to_hg19.narrowPeak")),
                  "Microglia":pybedtools.BedTool(os.path.join("/projects",
                                                              "pfenninggroup",
                                                              "machineLearningForComputationalBiology",
                                                              "eramamur_stuff",
                                                              "gwas_enrichments",
                                                              "roadmap_dnase",
                                                              "gosselin_microglia_exvivo_idr_optimal_peak_liftover_hg38_to_hg19.narrowPeak")),
                  "Neuron":pybedtools.BedTool(os.path.join("/projects",
                                                           "pfenninggroup",
                                                           "machineLearningForComputationalBiology",
                                                           "eramamur_stuff",
                                                           "gwas_enrichments",
                                                           "roadmap_dnase",
                                                           "fullard_neun_put_idr_optimal_peak_liftover_hg38_to_hg19.narrowPeak"))}
    

In [24]:
region_chrom = region_bed[0][0]
region_start = int(region_bed[0][1])
region_end = int(region_bed[0][2])
region_length = region_end-region_start

In [25]:
ref_X = np.load(os.path.join(rootPath,"jansen_snps_intersecting_alpk2_region_1_50kb_reference_X.npy"))
alt_X = np.load(os.path.join(rootPath,"jansen_snps_intersecting_alpk2_region_1_50kb_alternate_X.npy"))
snp_info = pd.read_csv(os.path.join(rootPath,"jansen_snps_intersecting_alpk2_region_1_50k_hg19_alleles.txt"),
                            sep='\t',
                            header=None,
                            names=["CHR", "START", "END", "SNP", "REF", "ALT", "P", "Z"])

In [26]:
snp_info

,CHR,START,END,SNP,REF,ALT,P,Z
0,chr18,56164536,56164537,rs4271673,C,T,0.599217,0.525527
1,chr18,56164694,56164695,18:56164695,A,G,0.548547,-0.599939
2,chr18,56164739,56164740,rs4432325,G,A,0.033950,2.120665
3,chr18,56164745,56164746,rs4616363,G,A,0.636187,0.473036
4,chr18,56165163,56165164,rs2277720,G,A,0.420584,-0.805409
...,...,...,...,...,...,...,...,...
269,chr18,56213617,56213618,rs4598995,A,C,0.682300,-0.409327
270,chr18,56213702,56213703,rs150927750,A,T,0.101300,-1.638584
271,chr18,56214008,56214009,rs62096278,G,A,0.714848,-0.365353
272,chr18,56214145,56214146,rs72937519,T,G,0.160260,1.404199


In [27]:
intersections = {}
for key in cell_type_beds:
    intersections[key] = pybedtools.bedtool.BedTool.intersect(cell_type_beds[key],
                                                              region_bed,
                                                              u=True
                                                             )

In [28]:
for key in bulk_models_list:
    model = bulk_models_list[key]
    ref_score = model.predict(ref_X)
    alt_score = model.predict(alt_X)
    effect_score = ref_score - alt_score
    snp_info[key+"_ref_score"] = ref_score
    snp_info[key+"_alt_score"] = alt_score
    snp_info[key+"_effect_score"] = effect_score

In [29]:
snp_info

,CHR,START,END,SNP,REF,ALT,P,Z,Microglia_ref_score,Microglia_alt_score,Microglia_effect_score,Monocyte_ref_score,Monocyte_alt_score,Monocyte_effect_score,Neuron_ref_score,Neuron_alt_score,Neuron_effect_score
0,chr18,56164536,56164537,rs4271673,C,T,0.599217,0.525527,5.166845,5.168638,-0.001793,4.655799,4.660488,-0.004688,6.993213,5.918385,1.074828
1,chr18,56164694,56164695,18:56164695,A,G,0.548547,-0.599939,6.403600,6.636279,-0.232678,3.607145,3.584226,0.022919,4.964695,4.859358,0.105337
2,chr18,56164739,56164740,rs4432325,G,A,0.033950,2.120665,3.772012,3.687018,0.084994,3.307685,3.346529,-0.038844,3.879764,3.668866,0.210898
3,chr18,56164745,56164746,rs4616363,G,A,0.636187,0.473036,4.152282,3.966160,0.186122,3.246958,3.215632,0.031326,3.866172,3.864683,0.001489
4,chr18,56165163,56165164,rs2277720,G,A,0.420584,-0.805409,4.807274,4.037744,0.769530,3.330312,2.872660,0.457652,6.398648,6.466461,-0.067813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,chr18,56213617,56213618,rs4598995,A,C,0.682300,-0.409327,6.596577,6.972142,-0.375565,4.835951,4.934411,-0.098460,7.188704,7.484279,-0.295575
270,chr18,56213702,56213703,rs150927750,A,T,0.101300,-1.638584,5.395319,5.664871,-0.269552,4.019012,3.920381,0.098631,5.289942,5.534971,-0.245029
271,chr18,56214008,56214009,rs62096278,G,A,0.714848,-0.365353,7.950653,8.013032,-0.062379,5.422115,5.275091,0.147024,3.944140,3.918724,0.025417
272,chr18,56214145,56214146,rs72937519,T,G,0.160260,1.404199,4.408566,4.484443,-0.075877,3.603268,3.680062,-0.076794,3.810853,3.885003,-0.074150


In [31]:
fig,axs = plt.subplots(3,1, figsize=(20,5))
fig.subplots_adjust(hspace=0)


for i,key in enumerate(intersections):
    print(i,key)
    for interval in intersections[key]:
        interval_start = int(interval[1])
        interval_end = int(interval[2])
        plot_start = 1.0*(interval_start - region_start)/(region_length)
        plot_end = 1.0*(interval_end - region_start)/(region_length)
        plot_start = max(plot_start,0)
        plot_end = min(plot_end,1)
        print(plot_start, plot_end)
        axs[i].axhline(y=20,xmin=plot_start,xmax=plot_end, lw=15, c=colorsDict[key])


    im = axs[i].scatter(snp_info["END"],
                        -np.log10(snp_info["P"]),
                        c=np.absolute(snp_info[key+"_effect_score"]),
                        cmap="Reds",
                        alpha=0.5,
                        s=50,
                        rasterized=True)
    axs[i].tick_params(axis='both', which='major', labelsize=12)
    axs[i].set_ylim([0,20])
    axs[i].set_xlim([region_start, region_end])
    axs[i].set_ylabel = key
    cb = fig.colorbar(im,ax=axs[i])
    cb.ax.tick_params(labelsize=12)


for i in range(len(axs)-1):
    axs[i].set_xticks([])
axs[len(axs)-1].set_xlabel("Genome position",size=20)
axs[len(axs)-1].ticklabel_format(style='plain', useOffset=False)

        
plt.tight_layout()
plt.savefig("alpk2_region_1_50kb_scores.svg")
plt.close()

0 Monocyte
0.01666 0.02078
0.07724 0.08218
0.47546 0.48176
0.498 0.50284
0.56704 0.57434
0.57632 0.58404
0.65146 0.65704
0.75432 0.77012
0.78774 0.79412
0.8899 0.90536
0.8899 0.90536
0.95544 0.9614
1 Microglia
0.7551 0.76878
0.78676 0.79432
0.07612 0.08714
0.89704 0.90488
0.11222 0.11794
0.9549 0.96184
0.91038 0.91642
0.6519 0.65644
0.57472 0.58322
0.37966 0.38678
0.6519 0.65644
0.91038 0.91642
0.68186 0.68592
0.82962 0.8349
0.38806 0.39476
0.89114 0.89598
0.38806 0.39476
0.7551 0.76878
0.07612 0.08714
0.7551 0.76878
2 Neuron
0.65188 0.65578
0.1581 0.17112
0.1581 0.17112


In [143]:
snp_info.sort_values(by="Monocyte_effect_score").head(50)

,CHR,START,END,SNP,REF,ALT,P,Z,Microglia_ref_score,Microglia_alt_score,Microglia_effect_score,Monocyte_ref_score,Monocyte_alt_score,Monocyte_effect_score,Neuron_ref_score,Neuron_alt_score,Neuron_effect_score
448,chr11,59994756,59994757,rs144834075,G,A,4.348129e-01,-0.780982,10.211926,11.135289,-0.923363,4.498094,6.807083,-2.308989,4.265267,6.521447,-2.256180
13,chr11,59887968,59887969,rs141088961,T,A,4.406000e-01,0.771180,1.952662,2.479453,-0.526790,4.265637,5.566752,-1.301115,3.053890,3.296562,-0.242672
186,chr11,59935937,59935938,rs117587394,A,G,1.134948e-01,1.582678,6.067342,6.256288,-0.188945,5.819202,7.103384,-1.284182,6.807069,6.936926,-0.129857
568,chr11,60021585,60021586,rs76608006,C,A,2.363857e-01,-1.184069,4.472965,6.013577,-1.540612,3.636696,4.882198,-1.245502,5.581674,5.543464,0.038210
564,chr11,60021483,60021484,rs4939327,A,T,1.876879e-01,-1.317450,6.537091,7.002129,-0.465037,4.956997,6.094333,-1.137336,5.452012,5.805986,-0.353974
468,chr11,60000299,60000300,rs668095,T,C,2.172992e-01,1.233742,8.676871,9.827815,-1.150944,4.091390,5.153425,-1.062035,6.413243,7.320018,-0.906775
3,chr11,59884240,59884241,rs1303617,T,G,1.665723e-01,-1.383302,8.996817,11.559979,-2.563163,4.005779,5.012404,-1.006625,3.953089,4.484591,-0.531502
518,chr11,60011796,60011797,rs11602454,A,G,1.642000e-01,-1.391084,4.485580,4.717942,-0.232362,3.194277,4.127940,-0.933664,3.389210,3.370209,0.019001
385,chr11,59980908,59980909,rs149822502,G,C,6.711000e-01,0.424639,8.564525,8.459477,0.105047,4.089123,5.018047,-0.928924,3.824317,3.739689,0.084629
365,chr11,59977229,59977230,rs582407,G,C,2.067281e-01,1.262614,3.671604,4.538377,-0.866774,3.961171,4.850688,-0.889517,4.251997,4.317658,-0.065661


In [ ]:
#rs636317
#rs76726049